In [1]:
import pandas as pd
import re


In [2]:
raw_df = pd.read_csv("Manually_Categorized.csv")

In [3]:
df = raw_df[["Project Name", "Start Date", "Amount Awarded", "Category"]].rename(columns = dict(Category = "cat0"))

In [4]:
keywords = dict(Medical = [], Food = [] ,Others = [])

In [5]:
def convert_to_non_special_cw_list(_str):
    return re.sub(r"[^A-Za-z0-9\s\-]", "", _str.lower()).split(" ")

In [6]:
def update_keywords(cycle):
    #extract keywords
    for cat in keywords.keys():
        cat_string = " ".join(df[df[f"cat{cycle}"] == cat]["Project Name"].str.lower())
        keywords[cat] = list(set(convert_to_non_special_cw_list(cat_string)))
        
    #remove intersection keywords
    temp_keywords = dict(keywords)
    for cat in keywords.keys():
        other_cats = list(set(keywords.keys()) - set([cat]))
        final_cat_keywords = set(temp_keywords[cat]) - set(temp_keywords[other_cats[0]]) - set(temp_keywords[other_cats[1]])
        keywords[cat] = list(final_cat_keywords)

In [7]:
def categorizer(name):
    name_list = convert_to_non_special_cw_list(name.lower())
    
    for cat in keywords.keys():
        if set(name_list).intersection(set(keywords[cat])):
            return cat
    else:
        if ("medical equipment" in name.lower()) or ("medical suppl" in name.lower()) \
        or ("hospital equipment" in name.lower()) or ("hospital suppl" in name.lower()):
            return "Medical"
        elif ("equipment" in name.lower()):
            return "Others"
        return "Not classified"

In [8]:
def percentage_calcu(category): 
    if category == "Not classified": 
        cat_total_amount_awarded = df[df.iloc[:,-2] == "Not classified"]["Amount Awarded"].sum()
    else: 
        cat_total_amount_awarded = df[df.iloc[:,-1] == category]["Amount Awarded"].sum()
    percent_budget_not_categorized = cat_total_amount_awarded/df["Amount Awarded"].sum() * 100
    return percent_budget_not_categorized
        

In [9]:
cycle = 0
previous_unknown_count = 0
while True:
    update_keywords(cycle)
    cycle += 1
    df[f"cat{cycle}"] = df[f"cat{cycle-1}"]
    df.loc[df[f"cat{cycle}"]=="Not classified", f"cat{cycle}"] = df.loc[df[f"cat{cycle}"]=="Not classified", "Project Name"].map(categorizer)
    current_unknown_count = (df[f"cat{cycle}"] == "Not classified").sum()
    print(current_unknown_count)
    if current_unknown_count == previous_unknown_count:
        break
    else:
        previous_unknown_count = current_unknown_count

2001
1300
1265
1262
1262


In [10]:
df["Final Categorization"] = df.iloc[:,-1].replace("Not classified", "Others")

In [11]:
list(map(percentage_calcu,["Not classified","Medical", "Food", "Others"]))

[1.6181937276321727, 64.99204259795222, 28.49387135145384, 6.514086050593932]

In [12]:
df.to_csv("auto-classified.csv")